Code to test the effectiveness of term similarity through Cosine Similarity metrics

In [182]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../')
import cleaner

Loading global tagger... please wait a few seconds
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [183]:
df_prods = pd.read_csv('../data/product_descriptions.csv')

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.corpus import stopwords


vect = CountVectorizer(min_df=1, stop_words=stopwords.words('english'), preprocessor=)
X_vect = vect.fit_transform(df_prods['product_description'])

In [ ]:
tfidf = TfidfVectorizer(min_df=1, stop_words=stopwords.words('english'))
X_tfidf = tfidf.fit_transform(df_prods['product_description'])

In [ ]:
# How to convert a sentance to a vector
# gives back a 1 x p vector and counts the occurences
print vect.transform(['common angles'])

In [ ]:
print tfidf.shape

# Comapre with the search term

In [ ]:
df_train = pd.read_csv('../data/train_sample.csv')

In [ ]:
vect.transform(df_train.head(n=1)['search_term'])

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:

row_num = 2
row = df_train.loc[row_num]
print row.relevance
print row['search_term']
print row['product_description']

In [ ]:
def get_cosine_similarity(row, df_corpus, vectorizer, X):
    row_num = df_corpus[df_corpus['product_uid'] == row['product_uid']].index
    return cosine_similarity(X[row_num],vectorizer.transform([row['search_term']])).tolist()[0][0]
df_train['cosine_count'] = df_train.apply(get_cosine_similarity, axis=1, args = (df_prods, vect, X_vect))
df_train['cosine_tfidf'] = df_train.apply(get_cosine_similarity, axis=1, args = (df_prods, tfidf, X_tfidf))

In [ ]:
df_train[['cosine_count', 'cosine_tfidf']]

In [ ]:
plt.scatter(df_train['cosine_tfidf'], df_train['relevance'] )

In [ ]:
ax = plt.subplot(111, projection='3d')
ax.scatter(df_train['cosine_tfidf'], df_train['cosine_count'],
               zs=df_train['relevance'], s=20, c='b', depthshade=True)

In [ ]:

row_num = df_prods[df_prods['product_uid'] == row['product_uid']].index



In [ ]:

row_num = df_prods[df_prods['product_uid'] == row['product_uid']].index
cosine_similarity(X_tfidf[row_num],tfidf.transform([row['search_term']]))

In [105]:
vect_singular = CountVectorizer(min_df=1, stop_words=stopwords.words('english'))
X_sing = vect_singular.fit_transform(row[['product_description', 'search_term']])
cosine_similarity(X_sing[0],X_sing[1])

array([[ 0.]])